In [1]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub

yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

2023-02-20 23:11:05.184305: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-20 23:11:09.231906: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# scan data directories
import glob

nature_file_list = []
nature_file_list += glob.glob('../data/interim/AmbisonicSoundLibrary/nature/*')
nature_file_list += glob.glob('../data/interim/GoogleAudioSet/Outside, rural or natural/*')
nature_file_list += glob.glob('../data/interim/youtube/NatureSoundscapes/*')
nature_file_list += glob.glob('../data/interim/youtube/NomadicAmbience_nature/*')
nature_file_list += glob.glob('../data/interim/S2L_LULC/non_urban/*')
nature_file_list += glob.glob('../data/interim/S2L_LULC/urban_0_25/*')

city_file_list = []
city_file_list += glob.glob('../data/interim/GoogleAudioSet/Outside, urban or manmade/*')
city_file_list += glob.glob('../data/interim/youtube/NomadicAmbience_city/*')
city_file_list += glob.glob('../data/interim/SONYC/**/*.pkl')
city_file_list += glob.glob('../data/interim/S2L_LULC/urban_26_100/*')

nature_source_list = ['nature_'+i.rsplit('/', 3)[1]+'/'+i.rsplit('/', 3)[2] for i in nature_file_list]
city_source_list = ['city_'+i.rsplit('/', -1)[3] for i in city_file_list]

In [3]:
nature_df = pd.DataFrame({'file': nature_file_list, 'source': nature_source_list, 'category': 0})
city_df = pd.DataFrame({'file': city_file_list, 'source': city_source_list, 'category': 1})
df_all = pd.concat([nature_df, city_df], ignore_index=True)
df_all

,file,source,category
0,../data/interim/AmbisonicSoundLibrary/nature/W...,nature_AmbisonicSoundLibrary/nature,0
1,../data/interim/AmbisonicSoundLibrary/nature/R...,nature_AmbisonicSoundLibrary/nature,0
2,../data/interim/AmbisonicSoundLibrary/nature/A...,nature_AmbisonicSoundLibrary/nature,0
3,../data/interim/AmbisonicSoundLibrary/nature/W...,nature_AmbisonicSoundLibrary/nature,0
4,../data/interim/AmbisonicSoundLibrary/nature/L...,nature_AmbisonicSoundLibrary/nature,0
...,...,...,...
1919,../data/interim/S2L_LULC/urban_26_100/s2lam111...,city_S2L_LULC,1
1920,../data/interim/S2L_LULC/urban_26_100/s2lam006...,city_S2L_LULC,1
1921,../data/interim/S2L_LULC/urban_26_100/s2lam083...,city_S2L_LULC,1
1922,../data/interim/S2L_LULC/urban_26_100/s2lam052...,city_S2L_LULC,1


In [4]:
# # randomly sampled 80% as training, 10% as testing, and 10% and validation
# test_df = df_all.groupby('source', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=23))
# val_df = test_df.groupby('source', group_keys=False).apply(lambda x: x.sample(frac=0.5, random_state=23))
# # drop the ones got splitted out
# train_df = df_all.drop(index = test_df.index)
# test_df = test_df.drop(index = val_df.index)

# test_df.reset_index(inplace=True, drop=True)
# train_df.reset_index(inplace=True, drop=True)
# val_df.reset_index(inplace=True, drop=True)

In [5]:
# from sklearn.model_selection import StratifiedKFold

# # Split the data into folds using StratifiedKFold
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=23)
# for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['source'])):
#     # Assign the fold number to each row in the DataFrame
#     train_df.loc[val_idx, 'fold'] = fold

In [6]:
from sklearn.model_selection import StratifiedKFold

# Split the data into folds using StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=23)
for fold, (train_idx, val_idx) in enumerate(skf.split(df_all, df_all['source'])):
    # Assign the fold number to each row in the DataFrame
    df_all.loc[val_idx, 'fold'] = fold
    
df_all['fold'] = df_all['fold'].astype('int')
df_all = df_all.sample(frac=1, random_state=23).reset_index(drop=True)

In [7]:
df_all

,file,source,category,fold
0,../data/interim/S2L_LULC/urban_0_25/s2llg003_1...,nature_S2L_LULC/urban_0_25,0,9
1,"../data/interim/GoogleAudioSet/Outside, urban ...",city_GoogleAudioSet,1,8
2,../data/interim/SONYC/audio-12/34_018803.pkl,city_SONYC,1,7
3,"../data/interim/GoogleAudioSet/Outside, urban ...",city_GoogleAudioSet,1,4
4,"../data/interim/GoogleAudioSet/Outside, urban ...",city_GoogleAudioSet,1,0
...,...,...,...,...
1919,../data/interim/SONYC/audio-4/22_004702.pkl,city_SONYC,1,1
1920,../data/interim/S2L_LULC/urban_0_25/s2lam042_1...,nature_S2L_LULC/urban_0_25,0,2
1921,"../data/interim/GoogleAudioSet/Outside, urban ...",city_GoogleAudioSet,1,1
1922,../data/interim/S2L_LULC/urban_0_25/s2llg001_1...,nature_S2L_LULC/urban_0_25,0,0


In [8]:
filenames = df_all['file']
targets = df_all['category']
folds = df_all['fold']

main_ds = tf.data.Dataset.from_tensor_slices((filenames, targets, folds))
main_ds.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [9]:
def load_wav_pkl(filename):
    import pickle
    # open a file, where you stored the pickled data
    file = open(filename, 'rb')

    # dump information to that file
    output = pickle.load(file)
    wav = output['y']

    # close the file
    file.close()
    return wav

In [10]:
wav_list = []
for index, row in df_all.iterrows():
    if len(load_wav_pkl(row['file'])) == 160000:
        wav_list.append(load_wav_pkl(row['file']))
    if len(load_wav_pkl(row['file'])) != 160000: # if the waveform is shorter (for unknown reason)
        temp_wav = load_wav_pkl(row['file'])
        print('shorter length: '+str(len(temp_wav)))
        wav_list.append(np.pad(temp_wav, (0,160000-len(temp_wav)),'mean')) # zero-padding at the end to 160000
    


shorter length: 159999
shorter length: 159880
shorter length: 146099
shorter length: 159880
shorter length: 159880
shorter length: 151683
shorter length: 159880
shorter length: 153357
shorter length: 153242
shorter length: 157848
shorter length: 159992
shorter length: 151461
shorter length: 148006
shorter length: 159997
shorter length: 156480
shorter length: 159993


In [11]:
main_ds = tf.data.Dataset.from_tensor_slices((np.stack(wav_list, axis = 0), df_all['category'], df_all['fold']))
main_ds.element_spec

(TensorSpec(shape=(160000,), dtype=tf.float32, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [12]:
# applies the embedding extraction model to a wav data
def extract_embedding(wav_data, label, fold):
    # run YAMNet to extract embedding from the wav data
    scores, embeddings, spectrogram = yamnet_model(wav_data)
    num_embeddings = tf.shape(embeddings)[0]
    return (embeddings,
            tf.repeat(label, num_embeddings),
            tf.repeat(fold, num_embeddings))

# extract embedding
main_ds = main_ds.map(extract_embedding).unbatch()
main_ds.element_spec

(TensorSpec(shape=(1024,), dtype=tf.float32, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [13]:
cached_ds = main_ds.cache()
train_ds = cached_ds.filter(lambda embedding, label, fold: fold < 8)
val_ds = cached_ds.filter(lambda embedding, label, fold: fold == 8)
test_ds = cached_ds.filter(lambda embedding, label, fold: fold == 9)

# remove the folds column now that it's not needed anymore
remove_fold_column = lambda embedding, label, fold: (embedding, label)

train_ds = train_ds.map(remove_fold_column)
val_ds = val_ds.map(remove_fold_column)
test_ds = test_ds.map(remove_fold_column)

train_ds = train_ds.cache().shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


# Customize model

In [14]:
my_classes = ['city', 'nature']
my_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1024), dtype=tf.float32,
                          name='input_embedding'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(len(my_classes))
], name='my_model')

my_model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               524800    
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 525,826
Trainable params: 525,826
Non-trainable params: 0
_________________________________________________________________


In [15]:
my_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 optimizer="adam",
                 metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                            patience=3,
                                            restore_best_weights=True)

In [16]:
history = my_model.fit(train_ds,
                       epochs=20,
                       validation_data=val_ds,
                       callbacks=callback)

Epoch 1/20
963/963 [==============================] - 134s 134ms/step - loss: 0.4224 - accuracy: 0.8106 - val_loss: 0.4727 - val_accuracy: 0.7995
Epoch 2/20
963/963 [==============================] - 6s 6ms/step - loss: 0.3689 - accuracy: 0.8395 - val_loss: 0.4839 - val_accuracy: 0.8031
Epoch 3/20
963/963 [==============================] - 6s 7ms/step - loss: 0.3337 - accuracy: 0.8578 - val_loss: 0.5007 - val_accuracy: 0.7961
Epoch 4/20
963/963 [==============================] - 8s 8ms/step - loss: 0.2997 - accuracy: 0.8706 - val_loss: 0.5255 - val_accuracy: 0.7958
Epoch 5/20
963/963 [==============================] - 9s 9ms/step - loss: 0.2754 - accuracy: 0.8847 - val_loss: 0.5299 - val_accuracy: 0.7945
Epoch 6/20
963/963 [==============================] - 8s 8ms/step - loss: 0.2766 - accuracy: 0.8916 - val_loss: 0.5533 - val_accuracy: 0.7930
Epoch 7/20
963/963 [==============================] - 7s 8ms/step - loss: 0.2377 - accuracy: 0.9024 - val_loss: 0.6086 - val_accuracy: 0.7911
Ep

In [17]:
loss, accuracy = my_model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

120/120 [==============================] - 2s 6ms/step - loss: 1.2178 - accuracy: 0.7622
Loss:  1.2177757024765015
Accuracy:  0.7622395753860474
